Fraction Fourier Transform based on: https://nalag.cs.kuleuven.be/research/software/FRFT/


In [35]:
%pip install numpy scipy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [36]:
import numpy as np
import scipy
import scipy.signal

In [37]:
# Matlab to Python translation from
# https://nalag.cs.kuleuven.be/research/software/FRFT/frft.m


def frft(f, a):
    ret = np.zeros_like(f, dtype=np.complex128)
    f = f.copy().astype(np.complex128)
    N = len(f)
    shft = np.fmod(np.arange(N) + np.fix(N / 2), N).astype(int)
    sN = np.sqrt(N)
    a = np.remainder(a, 4.0)

    if a == 0.0:
        return f
    if a == 2.0:
        return np.flipud(f)
    if a == 1.0:
        ret[shft] = np.fft.fft(f[shft]) / sN
        return ret
    if a == 3.0:
        ret[shft] = np.fft.ifft(f[shft]) * sN
        return ret

    # reduce to interval 0.5 < a < 1.5
    if a > 2.0:
        a = a - 2.0
        f = np.flipud(f)
    if a > 1.5:
        a = a - 1
        f[shft] = np.fft.fft(f[shft]) / sN
    if a < 0.5:
        a = a + 1
        f[shft] = np.fft.ifft(f[shft]) * sN

    # the general case for 0.5 < a < 1.5
    alpha = a * np.pi / 2
    tana2 = np.tan(alpha / 2)
    sina = np.sin(alpha)
    f = np.hstack((np.zeros(N - 1), sincinterp(f), np.zeros(N - 1))).T

    # chirp premultiplication
    chrp = np.exp(-1j * np.pi / N * tana2 / 4 * np.arange(-2 * N + 2, 2 * N - 1).T ** 2)
    f = chrp * f

    # chirp convolution
    c = np.pi / N / sina / 4
    ret = scipy.signal.fftconvolve(
        np.exp(1j * c * np.arange(-(4 * N - 4), 4 * N - 3).T ** 2), f
    )
    ret = ret[4 * N - 4 : 8 * N - 7] * np.sqrt(c / np.pi)

    # chirp post multiplication
    ret = chrp * ret

    # normalizing constant
    ret = np.exp(-1j * (1 - a) * np.pi / 4) * ret[N - 1 : -N + 1 : 2]

    return ret


def ifrft(f, a):
    return frft(f, -a)


def sincinterp(x):
    N = len(x)
    y = np.zeros(2 * N - 1, dtype=x.dtype)
    y[: 2 * N : 2] = x
    xint = scipy.signal.fftconvolve(
        y[: 2 * N],
        np.sinc(np.arange(-(2 * N - 3), (2 * N - 2)).T / 2),
    )

    return xint[2 * N - 3 : -2 * N + 3]

In [38]:
# Matlab to Python translation from
# https://nalag.cs.kuleuven.be/research/software/FRFT/frft2.m


def frft2(f, a):
    f0 = f.flatten()
    N = len(f)
    sN = np.sqrt(N)
    a = np.mod(a, 4)

    if a == 0:
        return f0
    elif a == 2:
        return np.flipud(f0)
    elif a == 1:
        return np.fft.ifftshift(np.fft.fft(np.fft.fftshift(f0))) / sN
    elif a == 3:
        return np.fft.ifftshift(np.fft.ifft(np.fft.fftshift(f0))) * sN

    if a > 2.0:
        a = a - 2
        f0 = np.flipud(f0)
    if a > 1.5:
        a = a - 1
        f0 = np.fft.ifftshift(np.fft.fft(np.fft.fftshift(f0))) / sN
    if a < 0.5:
        a = a + 1
        f0 = np.fft.ifftshift(np.fft.ifft(np.fft.fftshift(f0))) * sN

    alpha = a * np.pi / 2
    s = np.pi / (N + 1) / np.sin(alpha) / 4
    t = np.pi / (N + 1) * np.tan(alpha / 2) / 4
    Cs = np.sqrt(s / np.pi) * np.exp(-1j * (1 - a) * np.pi / 4)

    f1 = fconv(f0, np.sinc(np.arange(-(2 * N - 3), 2 * N - 1, 2) / 2), 1)
    f1 = f1[N : 2 * N]
    chrp = np.exp(-1j * t * np.arange(-N, N) ** 2)
    l0 = chrp[::2]
    l1 = chrp[1::2]
    f0 = f0 * l0
    f1 = f1 * l1
    chrp = np.exp(1j * s * np.arange(-(2 * N), 2 * N - 1) ** 2)
    e1 = chrp[::2]
    e0 = chrp[1::2]
    f0 = fconv(f0, e0, 0)
    f1 = fconv(f1, e1, 0)
    h0 = np.fft.ifft(f0 + f1)

    Faf = Cs * l0 * h0[N : 2 * N]

    return Faf


def fconv(x, y, c):
    N = len(np.concatenate((x.flatten(), y.flatten()))) - 1
    P = 2 ** np.ceil(np.log2(N)).astype(int)
    z = np.fft.fft(x, P) * np.fft.fft(y, P)

    if c != 0:
        z = np.fft.ifft(z)
        z = z[N::-1]

    return z

In [39]:
%matplotlib inline

from ipywidgets import interact
import ipywidgets as widgets
import matplotlib.pyplot as plt

def plot_func(fraction, rot, freq, width):
    signal = np.zeros((1024))
    signal[len(signal)//2+freq-width:len(signal)//2+freq+width+1] = 1
    trans_sig = frft(signal,fraction)
    trans_sig2 = frft2(signal,fraction)

    ax = plt.figure().add_subplot(projection='3d')
    ax.view_init(30, -rot*360/4+45, 0)

    # Prepare arrays x, y, z
    x = np.linspace(-trans_sig.shape[0]//2, trans_sig.shape[0]//2, trans_sig.shape[0])
    y = trans_sig.imag
    z = trans_sig.real

    x2 = np.linspace(-trans_sig2.shape[0]//2, trans_sig2.shape[0]//2, trans_sig2.shape[0])
    y2 = trans_sig2.imag
    z2 = trans_sig2.real

    ax.scatter(x, y, z, label='parametric curve',color="green", s=0.5)
    ax.plot(x, y, z, label='parametric curve',alpha=0.1,color="green")

    ax.scatter(x2, y2, z2, label='parametric curve',color="red", s=0.5)
    ax.plot(x2, y2, z2, label='parametric curve',alpha=0.1,color="red")

    ax.set_xlim([-205, 205])
    plt.axis('off')

x = interact(plot_func, fraction = widgets.FloatSlider(value=0.56,
                                               min=0,
                                               max=4,
                                               step=0.001),
                                    rot = widgets.FloatSlider(value=1.5,
                                               min=-4,
                                               max=4,
                                               step=0.001),
                    freq = widgets.IntSlider(value=0,
                                               min=-60,
                                               max=60,
                                               step=1),
                    width = widgets.IntSlider(value=20,
                                               min=0,
                                               max=60,
                                               step=1))

interactive(children=(FloatSlider(value=0.56, description='fraction', max=4.0, step=0.001), FloatSlider(value=…